In [ ]:
!pip -q install transformers emoji datasets sentencepiece simpletransformers

     |████████████████████████████████| 2.6 MB 33.7 MB/s 
     |████████████████████████████████| 185 kB 57.0 MB/s 
     |████████████████████████████████| 542 kB 52.1 MB/s 
     |████████████████████████████████| 1.2 MB 40.9 MB/s 
     |████████████████████████████████| 221 kB 56.2 MB/s 
     |████████████████████████████████| 895 kB 58.3 MB/s 
     |████████████████████████████████| 636 kB 50.4 MB/s 
     |████████████████████████████████| 3.3 MB 29.7 MB/s 
     |████████████████████████████████| 76 kB 5.5 MB/s 
     |████████████████████████████████| 118 kB 62.2 MB/s 
     |████████████████████████████████| 243 kB 61.2 MB/s 
     |████████████████████████████████| 43 kB 2.1 MB/s 
     |████████████████████████████████| 1.8 MB 41.7 MB/s 
     |████████████████████████████████| 7.9 MB 28.9 MB/s 
     |████████████████████████████████| 124 kB 61.6 MB/s 
     |████████████████████████████████| 133 kB 70.3 MB/s 
     |████████████████████████████████| 170 kB 61.3 MB/s 
     |████████████

In [ ]:
!gdown --id 1ogDMFgTH10S7wmXRShWJx2wle-r_5JGY
!unzip -qq -o 'ai-hackathon-online-2021-qa.zip'

!gdown --id 16arb_fdhb8xSfGBAqepTXKL3mPWtILh7
!unzip -qq -o 'AIFORTHAI-ThaiWikiQA_NSC2020.zip'
!unzip -qq -o 'wiki-documents-nsc.zip'

!gdown --id 1SPYEmxekBc0M7iSLXHXwTQVJxo2hRAPE

Downloading...
From: https://drive.google.com/uc?id=1ogDMFgTH10S7wmXRShWJx2wle-r_5JGY
To: /content/ai-hackathon-online-2021-qa.zip
100% 488k/488k [00:00<00:00, 7.69MB/s]
Downloading...
From: https://drive.google.com/uc?id=16arb_fdhb8xSfGBAqepTXKL3mPWtILh7
To: /content/AIFORTHAI-ThaiWikiQA_NSC2020.zip
171MB [00:01, 160MB/s]
Downloading...
From: https://drive.google.com/uc?id=1SPYEmxekBc0M7iSLXHXwTQVJxo2hRAPE
To: /content/FullData.csv
164MB [00:00, 185MB/s]


### **DATA**

In [ ]:
import pandas as pd
import re
import string
import emoji
import os
from sklearn.model_selection import train_test_split
from datasets import load_dataset, Dataset, list_datasets

pd.set_option('display.max_colwidth',1000)

In [ ]:
def strip_emoji(text):
    new_text = re.sub(emoji.get_emoji_regexp(), r"", text)
    return new_text

def remove_HTML_syntax(input_text):
    text = ''
    accumulate = False
    output_text = input_text
    for char in input_text:
        if char == '<':
            accumulate = True
        elif char == '>':
            output_text = output_text.replace(text + char, '')
            text = ''
            accumulate = False
        
        if accumulate:
            text += char

    return output_text

def clean_msg(msg):
     
    # ลบ text ที่อยู่ในวงเล็บ <> ทั้งหมด
    #msg = re.sub(r'<.*?>','', msg)
    
    msg = re.sub(r'--', '', msg)
    
    # ลบ เครื่องหมายคำพูด (punctuation)
    #for c in string.punctuation:
     #   if c != "#" or ":" or "/" or ".":
      #      msg = re.sub(r'\{}'.format(c),'',msg)
    
    # ลบ separator เช่น \n \t
    msg = ' '.join(msg.split())
    
    msg = strip_emoji(msg)
    msg = remove_HTML_syntax(msg)
    
    return msg

In [ ]:
def make_dict_data(df):
    data = []
    qas = []
    id = 1

    for idx, n in enumerate(df["context"]):
        same = True
        data_ = {"context": n.lower()}
        # qas
        
        next_ = idx + 1
        if next_ == len(df):
            next_ = idx -1
        if df["article_path"][idx] == df["article_path"][next_]:
            answer = {"id" : str(id),
                            "is_impossible": check_is_impossible(n, df["answer"][idx]),
                            "question": (df['question'][idx]).lower(),
                            "answers": [
                                {"text": (df["answer"][idx]).lower(),
                                "answer_start": n.find( df["answer"][idx])
                                }]
                            }                  
            id += 1
            qas.append(answer)
            continue
        
        else:
            answer = {"id" : str(id),
                            "is_impossible": check_is_impossible(n, df["answer"][idx]),
                            "question": (df['question'][idx]).lower(),
                            "answers": [
                                {"text": (df["answer"][idx]).lower(),
                                "answer_start": n.find( df["answer"][idx])
                                }]
                            }                       
            id += 1
            qas.append(answer)
            same = False
                
        data_["qas"] = qas
        qas = []
        data.append(data_)
    return data

def check_is_impossible(context, text):
  if context.find(text) == -1:
    return True
  else :
    return False

In [ ]:
all_df = pd.read_csv("FullData.csv")

In [ ]:
qa_data = all_df[all_df['answer_start'] != -1]

In [ ]:
qa_train_df, qa_valid_df = train_test_split(qa_data[:1000], test_size=0.4, random_state=1, shuffle=False)
qa_valid_df, qa_test_df = train_test_split(qa_valid_df, test_size=0.25, random_state=1, shuffle=False)

qa_train_df = qa_train_df.reset_index(drop=True)
qa_valid_df = qa_valid_df.reset_index(drop=True)
qa_test_df = qa_test_df.reset_index(drop=True)

NameError: ignored

In [ ]:
qa_train_data = make_dict_data(qa_train_df)
qa_valid_data = make_dict_data(qa_valid_df)
qa_test_data = make_dict_data(qa_test_df)

In [ ]:
test_df = pd.read_csv("Test.csv")
test_df

In [ ]:
def check_path(text):
  text = str(text)
  if text.find("SCG") >= 0:
    new_text = f"test/SCG{text[3:]}.txt"
    return new_text
  else:
    new_text = f"wiki-documents-nsc/{text}.txt"
    return new_text

def read_data(name_file: str):
    with open(name_file, "r", encoding="utf-8-sig") as file:
        text = file.read()
        file.close()
    msg = clean_msg(text)
    return msg

def do_path(path):
    new_path = path[:6] + "SCG" + path[6:]
    return new_path

def make_dict_prediction(df):
    data = []
    qas = []
    id = 1

    for idx, n in enumerate(df["context"]):
        data_ = {"context": n.replace("\xa0", "")}
        # qas
        qas_ = [{
            "question" : df['question'][idx].replace("\xa0", ""),
            "id" : id
        }]
        data_["qas"] = qas_
        qas = []
        id += 1
        data.append(data_)
    return data

In [ ]:
test_df['path'] = test_df['article_id'].map(lambda x : check_path(x))
test_df['context'] = test_df['path'].map(lambda x : read_data(x))

In [ ]:
test_data = make_dict_prediction(test_df)

**Model**

In [ ]:
#transformers
from transformers import (
    AutoConfig,
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    TrainingArguments,
    Trainer,
    default_data_collator,
)

In [ ]:
import torch
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [ ]:
model_args = {
    'manual_seed' : 0,
    'reprocess_input_data': False,

    'train_batch_size': 32,
    'eval_batch_size' : 64,

    'use_early_stopping': True,
    'early_stopping_delta': 0.01,
    'early_stopping_metric': 'eval_loss',
    'early_stopping_metric_minimize': True,
    'early_stopping_patience': 5,

    'evaluate_during_training' : True,
    'evaluate_during_training_verbose': True,

    'fp16': True,

    'num_train_epochs': 3,

    'overwrite_output_dir': True,

    'save_model_every_epoch': False,
    'save_steps':-1,

    'use_cached_eval_features' : True, 

    'max_seq_length': 256,
    'no_cache': False,

    'custom_parameter_groups':[{
        'params': ['classifier.weight', 'bert.encoder.layer.10.output.dense.weight'],
        'lr': 1e-5,
    }]
}

In [ ]:
bert = QuestionAnsweringModel(
    'bert', 
    'cstorm125/bert-base-multilingual-cased-finetune-qa',
    use_cuda=torch.cuda.is_available(), 
    args=model_args
)

Downloading:   0%|          | 0.00/692 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/398 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

In [ ]:
step, train_result = bert.train_model(qa_train_data, eval_data=qa_valid_data, show_running_loss=True)

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/258 [00:00<?, ?it/s]


convert squad examples to features:   0%|          | 0/300 [00:00<?, ?it/s]

In [ ]:
eval_result, eval_texts = bert.eval_model(qa_valid_data)

In [ ]:
eval_result

In [ ]:
test_result, test_texts = bert.eval_model(test)

In [ ]:
test_result

In [ ]:
answers, probabilities = bert.predict(test_data)

add example index and unique id: 100%|██████████| 5608/5608 [00:00<00:00, 252931.92it/s]


Running Prediction:   0%|          | 0/467 [00:00<?, ?it/s]

In [ ]:
answers

[{'id': 1,
  'answer': ['พลเอก ประยุทธ์ จันทร์โอชา (',
   'วิเชียร ตันติพิมลพันธุ์ น',
   'สุระชัย ถวิลไพร เป็น',
   'ประยุทธ์ จันทร์โอชา (',
   'วิเชียร ตันติพิมลพันธุ์ นักแต่งเพลงประกอบละครชื่อดัง เป็นผู้เรียบเรียงเนื้อร้อง ทำนองและเรียบเรียงมี พันตรี สุระชัย ถวิลไพร ข',
   'ประยุทธ์ จันทร์โอชา พลเอก ประยุทธ์ จันทร์โอชา (',
   'พันตรี สุระชัย ถวิลไพร ข',
   'พ',
   'ถวิลไพร เป็น',
   'สุระชัย ถวิลไพร เป็นผู้เรียบเรียงและทำนอง และมี จ่าสิบเอก พงศธร พอจิต เป็น',
   'สุระชัย ถ',
   'พันตรี สุระชัย ถวิลไพร เป็น',
   'พันตรี สุระชัย ถวิลไพร ขับร้องโดย จ่าสิบเอก พงศธร พอจิต ซ',
   'จ่าสิบเอก พงศธร พอจิต เป็น',
   'พลเอก ประยุทธ์ จันทร์โอชาส',
   'วิเชียร ตันติพิมลพันธุ์ นักแต่งเพลงประกอบละครชื่อดัง เป็น',
   'ประยุทธ์ จันทร์โอชา พ',
   'ส',
   'วิเชียร ตันติพิมลพันธุ์ นักแต่งเพลงประกอบละครชื่อดัง เป็นผู้เรียบเรียงเนื้อร้อง ทำนองและเรียบเรียงมี พันตรี ส',
   'ถวิลไพร เป็นผู้เรียบเรียงและทำนอง และมี จ่าสิบเอก พงศธร พอจิต เป็น']},
 {'id': 2,
  'answer': ['แมวเปอร์เซียและแมวขนสั้นเป็น',
   '',